# Continuous Control

---

In this notebook, you will learn how to use the Unity ML-Agents environment for the second project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893) program.

### 1. Start the Environment

We begin by importing the necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [1]:
from unityagents import UnityEnvironment
import numpy as np


Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Reacher.app"`
- **Windows** (x86): `"path/to/Reacher_Windows_x86/Reacher.exe"`
- **Windows** (x86_64): `"path/to/Reacher_Windows_x86_64/Reacher.exe"`
- **Linux** (x86): `"path/to/Reacher_Linux/Reacher.x86"`
- **Linux** (x86_64): `"path/to/Reacher_Linux/Reacher.x86_64"`
- **Linux** (x86, headless): `"path/to/Reacher_Linux_NoVis/Reacher.x86"`
- **Linux** (x86_64, headless): `"path/to/Reacher_Linux_NoVis/Reacher.x86_64"`

For instance, if you are using a Mac, then you downloaded `Reacher.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Reacher.app")
```

In [2]:
env = UnityEnvironment(file_name='Reacher.app')

UnityTimeOutException: The Unity environment took too long to respond. Make sure that :
	 The environment does not need user interaction to launch
	 The Academy and the External Brain(s) are attached to objects in the Scene
	 The environment and the Python interface have compatible versions.

Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [ ]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

In this environment, a double-jointed arm can move to target locations. A reward of `+0.1` is provided for each step that the agent's hand is in the goal location. Thus, the goal of your agent is to maintain its position at the target location for as many time steps as possible.

The observation space consists of `33` variables corresponding to position, rotation, velocity, and angular velocities of the arm.  Each action is a vector with four numbers, corresponding to torque applicable to two joints.  Every entry in the action vector must be a number between `-1` and `1`.

Run the code cell below to print some information about the environment.

In [ ]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agent and receive feedback from the environment.

Once this cell is executed, you will watch the agent's performance, if it selects an action at random with each time step.  A window should pop up that allows you to observe the agent, as it moves through the environment.  

Of course, as part of the project, you'll have to change the code so that the agent is able to use its experience to gradually choose better actions when interacting with the environment!

In [ ]:
env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
states = env_info.vector_observations                  # get the current state (for each agent)
scores = np.zeros(num_agents)                          # initialize the score (for each agent)
while True:
    actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
    actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
    env_info = env.step(actions)[brain_name]           # send all actions to tne environment
    next_states = env_info.vector_observations         # get next state (for each agent)
    rewards = env_info.rewards                         # get reward (for each agent)
    dones = env_info.local_done                        # see if episode finished
    scores += env_info.rewards                         # update the score (for each agent)
    states = next_states                               # roll over states to next time step
    if np.any(dones):                                  # exit loop if episode finished
        break
print('Total score (averaged over agents) this episode: {}'.format(np.mean(scores)))

When finished, you can close the environment.

In [ ]:
env.close()

### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```

In [ ]:
from ddpg_agent import Agent, ReplayBuffer, GaussianNoise
from ddpg_model import Actor, Critic
from ddpg_trainer import DDPGTrainer
from torch import optim
import torch
import time

from unityagents import UnityEnvironment
import numpy as np
env = UnityEnvironment(file_name='Reacher.app')

SEED = 42
device = 'cuda:0' if torch.cuda.is_available() else 'cpu:0'

In [ ]:
# Extract state dimensionality from env.
brain_name = env.brain_names[0]
env_info = env.reset(train_mode=True)[brain_name]
state_size = env_info.vector_observations.shape[-1]

# Extract action dimensionality and number of agents from env.
brain = env.brains[brain_name]
action_size = brain.vector_action_space_size
num_agents = len(env_info.agents)

# Display relevant environment information.
print(f'Number of agents: {num_agents},'
      + f'state size: {state_size} '
      + f', action size: {action_size}')

In [ ]:
# Initialize the replay buffer from which experiences are gathered for
# training the agent.
replay_buffer = ReplayBuffer(
    seed=SEED,
    buffer_size=int(1e5),
    batch_size=64
)

In [ ]:
# Initialize local and target Actor Networks and optimizer.
actor_local = Actor(state_size, action_size, SEED,
                    400, 300).to(device)
actor_target = Actor(state_size, action_size, SEED,
                        400, 300).to(device)
actor_optimizer = optim.Adam(actor_local.parameters(), lr=.000025)

# Initialize local and target Critic Networks and optimizer.
critic_local = Critic(state_size, action_size, SEED,
                        400, 300).to(device)
critic_target = Critic(state_size, action_size, SEED,
                        400, 300).to(device)
critic_optimizer = optim.Adam(critic_local.parameters(), lr=0.00025,
                                weight_decay=0)
# Initialize Gaussian noise to reduce generalization error.
noise = GaussianNoise(action_size, SEED, mu=0.0, sigma=0.3)

# Create agent object used for training.
agent = Agent(
    seed=SEED,
    memory=replay_buffer,
    batch_size=128,
    actor_local=actor_local,
    actor_target=actor_target,
    actor_optimizer=actor_optimizer,
    critic_local=critic_local,
    critic_target=critic_target,
    critic_optimizer=critic_optimizer,
    noise=noise,
    gamma=0.99,
    tau=0.001
)

# Initialize DDPGTrainer object with relevant arguments.
trainer = DDPGTrainer(
    env=env,
    agent=agent,
    update_frequency=1,
    num_updates=5,
    max_episode_length=1000,
    save_dir='save_dir',
    score_window_size=100
)

In [ ]:
n_episodes = 30
score_window_size = 100
target_score = 40
# Initialize timer and notify of training start.
t_start = time.time()
print('Starting training...')

# Train the agent for n_episodes.
for i in range(1, n_episodes + 1):

    # Step through the training process and notify progress
    trainer.train_step()
    trainer.print_status(put_new_line=False)

    # After every 10 episodes, initiate new line of notification.
    if trainer.i_episode % 10 == 0:
        trainer.print_status(put_new_line=True)

    # If mean score over last score_window_size episodes exceeds
    # target score, plot training progress and save trainer
    if len(trainer.scores) > score_window_size and \
            trainer.get_running_mean_score() > target_score:
        trainer.plt_mavg()
        print('\nEnvironment is solved.')
        print('Saving trainer...')
        trainer.save()
        print('Done.')
        break

# Close environment and notify of training time.
print('\nFinished training, closing env')
env.close()
t_end = time.time()
delta = t_end - t_start
minutes = delta / 60
print(f'Training took {minutes:.1f} minutes')